### Importing Packages

In [2]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate, load_prompt, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv

load_dotenv()

True

### Model Setup

In [3]:
model = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.7,
    max_tokens=700
)

### Static & Dynamic Prompts

In [4]:
result = model.invoke("Summarize word2vec paper in 5 lines.")
print(result.content)

The word2vec paper introduces two architectures: Continuous Bag-of-Words (CBOW) and Continuous Skip-Gram. CBOW predicts a word based on its context, while Skip-Gram predicts the context based on a word. Both models learn vector representations of words, capturing semantic relationships. The vectors are learned by optimizing the prediction task using stochastic gradient descent. The resulting word embeddings exhibit linear structure, enabling analogical reasoning.


In [5]:
template = PromptTemplate(
    template="""Please summarize the research paper titled {paper_input} with the following specifications:
        Explanation Style: {style_input}  
        Explanation Length: {length_input}  
        1. Mathematical Details: 
           - Include relevant mathematical equations if present in the paper.  
           - Explain the mathematical concepts using simple, intuitive code snippets where applicable.
        2. Analogies:
           - Use relatable analogies to simplify complex ideas.
        If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
        Ensure the summary is clear, accurate, and aligned with the provided style and length.
    """,
    input_variables=["paper_input","style_input","length_input"],
    validate_template=True
)

In [6]:
prompt = template.invoke({
    'paper_input': "word2vec: Efficient Estimation of Word Representations in Vector Space",
    'style_input': "Technical",
    'length_input': "5 lines"
})

In [7]:
result = model.invoke(prompt)
print(result.content)

The paper "word2vec: Efficient Estimation of Word Representations in Vector Space" proposes two architectures for learning word representations: Continuous Bag-of-Words (CBOW) and Skip-Gram.

**Key Points (5 lines):**
The CBOW architecture predicts a target word based on its context words, while Skip-Gram predicts the context words based on the target word. 
The objective function for Skip-Gram is: `L = Σ log(p(w_O|w_I))` where `w_O` is the output word and `w_I` is the input word. 
This is optimized using Negative Sampling: `p(w_O|w_I) = σ(v_{w_O}^T v_{w_I}) \* ∏[1 - σ(v_{w_N}^T v_{w_I})]` where `w_N` are negative samples. 
Mathematically, this can be represented as a dot product of word vectors: `v_{w_O}^T v_{w_I}`. 
The learned word representations capture semantic relationships, e.g., `v("king") - v("man") + v("woman") ≈ v("queen")`, illustrating the vector space's ability to encode analogies.

**Analogy:** Think of word vectors as points on a map, where semantically similar words a

### Prompt Reusability

In [36]:
template.save("template.json")

In [18]:
template = load_prompt("template.json")

### Template and Model Chain

In [8]:
chain = template | model

In [9]:
result = chain.invoke({
    'paper_input': "word2vec: Efficient Estimation of Word Representations in Vector Space",
    'style_input': "Beginner-friendly",
    'length_input': "5 lines"
})
print(result.content)

Here's a beginner-friendly summary of the "word2vec: Efficient Estimation of Word Representations in Vector Space" paper in 5 lines:

The word2vec paper introduces a method to represent words as vectors in a high-dimensional space, capturing their semantic meanings. It uses two architectures: Continuous Bag of Words (CBOW) and Skip-Gram. The Skip-Gram model predicts surrounding words given a center word, maximizing the objective function: `1/T * Σ log p(w_t+j | w_t)`, where `p(w_t+j | w_t) = exp(v_w_t * v_w_t+j) / Σ exp(v_w_t * v_w')`. Think of it like a word predictor game, where the model learns to identify likely neighboring words. The resulting vector representations can be used for tasks like text classification and clustering.

Mathematical Concept: The probability `p(w_t+j | w_t)` is calculated using the dot product of word vectors (`v_w_t * v_w_t+j`), representing the similarity between words.

Analogy: Imagine a map where similar words are placed near each other, like cities w

### Chatbot

In [10]:
chat_history = [
    SystemMessage(content='You are a helpful AI assistant')
]

In [11]:
while True:
    user_input = input('You: ')
    chat_history.append(HumanMessage(content=user_input))
    if user_input == 'exit':
        break
    result = model.invoke(chat_history)
    chat_history.append(AIMessage(content=result.content))
    print("AI: ",result.content)
print(chat_history)


[SystemMessage(content='You are a helpful AI assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='exit', additional_kwargs={}, response_metadata={})]


### Chat Prompt Template

In [12]:
chat_template = ChatPromptTemplate([
    ('system',"You are a helpful {domain} expert"),
    ('human',"Explain in simple terms, what is {topic}")
    # SystemMessage(content="You are a helpful {domain} expert"),
    # HumanMessage(content="Explain in simple terms, what is {topic}")
])

In [13]:
prompt = chat_template.invoke({
    'domain': 'AI',
    'topic': 'tokenization'
})

### Message Placeholders

In [14]:
chat_template = ChatPromptTemplate([
    ('system','You are a helpful customer support agent'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human','{query}')
])

In [ ]:
chat_history = []

with open('chat_history.txt') as f:
    chat_history.extend(f.readlines())

In [18]:
prompt = chat_template.invoke({'chat_history':chat_history, 'query':'Where is my refund'})